## Interactive Dashboard
An interactive dashboard is produced to display the results of the analysis on crop yield in Europe. A choropleth is displayed for Europe, with the user being able to switch between countries and year.

Note: I found that, in order to run this repeatedly, the kernal needs to be restarted.

In [ ]:
# Import libraries
import pandas as pd
import pycountry

In [ ]:
df = pd.read_csv('cropProduction.csv')

In [ ]:


def getCodes(countries):
    codes = []
    
    for country in countries:
        country_obj = pycountry.countries.get(name=country)
        
        # Extract the alpha-3 code from the country object
        code = country_obj.alpha_3
          
        codes.append(code)
    return codes 

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import dash
from dash import html
from dash import dcc


# Data
cols_dd = ["Barley", "Wheat", "Potatoes"]

app = dash.Dash()

app.layout = html.Div([
    dcc.Dropdown(
        id='demo-dropdown',
        options=[{'label': k, 'value': k} for k in cols_dd],
        value=cols_dd[0]
    ),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in range(2003,2021)],
        value=2019
    ),
    html.Hr(),
    dcc.Graph(id='display-selected-values'),

])

@app.callback(
    dash.dependencies.Output('display-selected-values', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value'),
     dash.dependencies.Input('year-dropdown', 'value')])

def update_output(crop,year):
    yieldd = df[(df["Element"] == 'Yield') & (df['Item'] == crop) & (df['Year'] == year)]

    fig = px.choropleth(yieldd,
                         locations=getCodes(yieldd['Area']),
                         color='Value',
                         hover_name='Area',
                         width=1600,height=800)
                        
    fig.update_layout(
        title_text=crop+' yield in '+str(year),
        title_x=0.5,
        geo_scope='europe',
       
        autosize=True)
    
    return fig

if __name__ == '__main__':
    app.run_server()